# Introduction

For this analysis, my research question would be what factors are associated with COVID-19 infection rates in United States counties. I would be utilizing various datasets concerning county health information, county COVID-19 infection rates, county population densities, and state political affiliation. In turn, I would attempt to identify and evaluate risk factors connected to infection rates by doing multiple regression analysis. The resultant linear regression model would be used mainly for interpretation. The outcome of the study, if deemed to be insightful and significant, can be used to access a population's vulnerability to COVID-19 based on the community's characteristics from the reference point of United States counties.  

In [42]:
# import necessary libraries 


import pandas as pd
import numpy as np 
import altair as alt
import statsmodels.formula.api as smf

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

# Data overview

Let's take a look at the datasets this analysis will be using. A few of them  are put together by crawling wiki pages. The rest are from what are provided officially.

In [43]:
# https://github.com/nytimes/covid-19-data
# Cumulative counts of coronavirus cases in the US at the county level
county_infection = pd.read_csv('relevant_data/us-counties.csv')

In [44]:
county_infection.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [45]:
county_infection.tail()

,date,county,state,fips,cases,deaths
126830,2020-05-08,Sublette,Wyoming,56035.0,3,0
126831,2020-05-08,Sweetwater,Wyoming,56037.0,19,0
126832,2020-05-08,Teton,Wyoming,56039.0,98,1
126833,2020-05-08,Uinta,Wyoming,56041.0,9,0
126834,2020-05-08,Washakie,Wyoming,56043.0,7,0


> The latest date of the data is May 8th, 2020.

In [47]:
# https://en.wikipedia.org/wiki/County_(United_States)
# County population and density
county_population = pd.read_csv('relevant_data/county-population.csv')

In [48]:
county_population.head()

,state,county,population,land_area_km,density_km
0,Alabama,Autauga,54571,1540,35.436
1,Alabama,Baldwin,182265,4118,44.261
2,Alabama,Barbour,27457,2292,11.979
3,Alabama,Bibb,22915,1612,14.215
4,Alabama,Blount,57322,1670,34.325


In [49]:
# https://en.wikipedia.org/wiki/Political_party_strength_in_U.S._states
# State party affiliation based on house representation
state_party_line = pd.read_csv('relevant_data/state_party_line.csv')

In [50]:
state_party_line.head()

,state,state_house_blue_perc
0,Alabama,22.86
1,Alaska,35.00
2,Arizona,43.33
3,Arkansas,25.71
4,California,72.50


In [51]:
# Source: https://www.countyhealthrankings.org/
# Access: https://app.namara.io/#/data_sets/579ee1c6-8f66-418c-9df9-d7b5b618c774?organizationId=5ea77ea08fb3bf000c9879a1
# County health information
county_health = pd.read_csv('relevant_data/us-county-health-rankings-2020.csv')

In [52]:
county_health.head()

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
0,1000,Alabama,NaN,81791.0,9942.794666,9840.535949,10045.053384,NaN,NaN,NaN,...,4.443264,3197324,65.413428,48517,1.061048,1.006759,1.115337,51.633032,1957932.0,40.963183
1,1001,Alabama,Autauga,791.0,8128.591190,7283.340731,8973.841649,1.0,NaN,NaN,...,2.965774,41316,74.308016,426,0.820225,0.347891,1.292558,51.448715,22921.0,42.002162
2,1003,Alabama,Baldwin,2967.0,7354.122530,6918.554269,7789.690790,1.0,NaN,NaN,...,4.646779,181201,83.111337,1068,0.543517,0.347271,0.739763,51.538377,77060.0,42.279099
3,1005,Alabama,Barbour,472.0,10253.573403,8782.217281,11724.929524,2.0,NaN,NaN,...,4.276355,11356,45.641252,398,1.631683,0.824903,2.438462,47.216752,18613.0,67.789635
4,1007,Alabama,Bibb,471.0,11977.539484,10344.064842,13611.014126,3.0,NaN,NaN,...,2.625000,16708,74.589286,57,0.268210,0.000000,0.807504,46.781250,15663.0,68.352607


In [53]:
county_health.columns[:75]

Index(['fips', 'state', 'county', 'num_deaths',
       'years_of_potential_life_lost_rate', '95percent_ci_low',
       '95percent_ci_high', 'quartile', 'ypll_rate_aian',
       'ypll_rate_aian_95percent_ci_low', 'ypll_rate_aian_95percent_ci_high',
       'ypll_rate_asian', 'ypll_rate_asian_95percent_ci_low',
       'ypll_rate_asian_95percent_ci_high', 'ypll_rate_black',
       'ypll_rate_black_95percent_ci_low', 'ypll_rate_black_95percent_ci_high',
       'ypll_rate_hispanic', 'ypll_rate_hispanic_95percent_ci_low',
       'ypll_rate_hispanic_95percent_ci_high', 'ypll_rate_white',
       'ypll_rate_white_95percent_ci_low', 'ypll_rate_white_95percent_ci_high',
       'percent_fair_or_poor_health', '95percent_ci_low_2',
       '95percent_ci_high_2', 'quartile_2',
       'average_number_of_physically_unhealthy_days', '95percent_ci_low_3',
       '95percent_ci_high_3', 'quartile_3',
       'average_number_of_mentally_unhealthy_days', '95percent_ci_low_4',
       '95percent_ci_high_4', 'quar

> This dataset contains extensive information about a county's health, including the rankings, quantiles, rates, and percentages of numerous demographic as well as health qualities. Of the many measurements of each quality, we probably only need one or two to avoid duplication. In addition, I will do a **factor analysis** on the columns to see if it makes sense.

For more information about these columns, please visit this [info](https://app.namara.io/#/data_sets/579ee1c6-8f66-418c-9df9-d7b5b618c774/info?organizationId=5ea77ea08fb3bf000c9879a1) page

# Data wrangling

In this section, we want to prepare our data for further exploration and analysis. 

In [54]:
# aggregate data related to county infection and basic characteristics
county = county_infection.merge(
    county_population, left_on=['county', 'state'], right_on=['county', 'state']
).merge(
    state_party_line, left_on=['state'], right_on=['state']
)

In [55]:
county.sample(5)

,date,county,state,fips,cases,deaths,population,land_area_km,density_km,state_house_blue_perc
11746,2020-03-19,Columbia,Wisconsin,55021.0,4,0,56833,1983,28.660,42.42
104875,2020-04-29,Taos,New Mexico,35055.0,17,0,32917,5706,5.769,61.90
102428,2020-04-09,Attala,Mississippi,28007.0,17,0,19564,1904,10.275,30.77
79290,2020-03-29,Prince George,Virginia,51149.0,4,0,37862,689,54.952,52.50
14410,2020-04-18,El Paso,Texas,48141.0,505,8,840410,2624,320.278,38.71


Let's look at the statistics of the counted days for the counties

In [63]:
def count_days(series):
    time_series = pd.to_datetime(series)
    first_date = time_series.iloc[0]
    last_date = time_series.iloc[-1]
    
    return (last_date - first_date).days + 1

In [64]:
grouped_county = county.groupby(['state', 'county']).agg(days_counted=('date', count_days))

In [65]:
grouped_county.describe()

,days_counted
count,2750.000000
mean,43.852000
std,12.192866
min,1.000000
25%,38.000000
50%,45.000000
75%,51.000000
max,109.000000


In [68]:
grouped_county.shape

(2750, 1)

We have 2750 counties in the data. The minimum amount of days counted for a county is only one, while the maximum is about two and a half months. I am happy that the median is a month and a half. Ideally, I want all counties in the analysis to have at least two months worth of data so that any of its heath characteristics can have a decent chance of exerting its influence if there is any at all. With the current data and analysis, I will only include counties with at least a month and a half worth of data to maximize the representativeness of the eventual infection picture and not exclude too much data. Please understand that I'm not a domain expert and don't have access to any. I apologize that this cutoff point seems rather arbitrary, but I hope the rationale at least somewhat makes sense domain-wise.

With that said, for the next step, we want to group the infection data by counties and create a bunch of aggregated columns including counted days, confirmed infection in the percentage of county population, death rate, and raw infection counts. We will also calculate those columns for the cutoff point of 45 days so that we can do the analysis without accounting for the number of days for model simplicity.

In [76]:
def county_cumulative_days(series, days = 45):
    if len(series) < days:
        return series.iloc[-1]
    else:
        return series.iloc[days - 1]

In [77]:
def group_data(data):
    grouped_data = data.groupby(['state', 'county']).agg(
        population=('population', lambda x: x.iloc[-1]),
        density_km=('density_km', lambda x: x.iloc[-1]),
        state_house_blue_perc=('state_house_blue_perc', lambda x: x.iloc[-1]),
        days_counted=('date', count_days),
        case_sum=('cases', lambda x: x.iloc[-1]),
        death_sum=('deaths', lambda x: x.iloc[-1]),
        case_count_45_days=('cases', county_cumulative_days),
        death_count_45_days=('deaths', county_cumulative_days)
    )
    
    grouped_data = grouped_data[grouped_data['days_counted'] >= 45]
    grouped_data['infection_rate'] = grouped_data['case_sum']/grouped_data['population']*100
    grouped_data['death_rate'] = grouped_data['death_sum']/grouped_data['case_sum']*100
    grouped_data = grouped_data[grouped_data['infection_rate'] != float("inf")]
    grouped_data['infection_rate_45_days'] = grouped_data['case_count_45_days']/grouped_data['population']*100
    grouped_data['death_rate_45_days'] = grouped_data['death_count_45_days']/grouped_data['case_count_45_days']*100
    
    return grouped_data.reset_index()

In [78]:
grouped_county = group_data(county)

In [80]:
grouped_county.sample(5)

,state,county,population,density_km,state_house_blue_perc,days_counted,case_sum,death_sum,case_count_45_days,death_count_45_days,infection_rate,death_rate,infection_rate_45_days,death_rate_45_days
1237,Texas,Castro,7843,3.372,38.71,49,22,1,18,1,0.280505,4.545455,0.229504,5.555556
107,California,Mono,13822,1.753,72.50,46,29,1,28,1,0.209810,3.448276,0.202576,3.571429
739,Mississippi,Sunflower,29450,16.388,30.77,48,63,3,60,3,0.213922,4.761905,0.203735,5.000000
628,Michigan,Ingham,280895,193.321,42.11,58,563,16,395,9,0.200431,2.841918,0.140622,2.278481
418,Indiana,Rush,17392,16.454,20.00,45,47,2,47,2,0.270239,4.255319,0.270239,4.255319


Next, let's tackle county health data.

Take a quick look over the data again.

In [86]:
county_health.sample(5)

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
2184,40033,Oklahoma,Cotton,126.0,12215.101821,8949.538842,15480.664800,4.0,NaN,NaN,...,8.292936,4387,75.952216,0,0.000000,0.000000,1.100292,50.831025,3687.0,59.534959
2678,48223,Texas,Hopkins,580.0,8562.906936,7500.792572,9625.021300,3.0,NaN,NaN,...,17.397446,26820,72.860636,992,2.924528,2.168209,3.680848,50.763380,20965.0,59.625722
2511,47081,Tennessee,Hickman,494.0,10729.289802,9283.688174,12174.891430,3.0,NaN,NaN,...,2.621394,22592,90.140845,68,0.291570,0.000000,0.811570,47.444440,24690.0,100.000000
2361,45009,South Carolina,Bamberg,326.0,12927.403725,10580.680489,15274.126961,4.0,NaN,NaN,...,2.227671,5153,36.098074,17,0.122302,0.000000,0.726117,51.901926,8714.0,54.506787
779,18135,Indiana,Randolph,428.0,10621.410664,9012.922419,12229.898908,4.0,NaN,NaN,...,3.533057,23319,93.835258,192,0.815079,0.009428,1.620730,50.899360,16191.0,61.866188


In [85]:
county_health.columns

Index(['fips', 'state', 'county', 'num_deaths',
       'years_of_potential_life_lost_rate', '95percent_ci_low',
       '95percent_ci_high', 'quartile', 'ypll_rate_aian',
       'ypll_rate_aian_95percent_ci_low',
       ...
       'percent_hispanic', 'num_non_hispanic_white',
       'percent_non_hispanic_white', 'num_not_proficient_in_english',
       'percent_not_proficient_in_english', '95percent_ci_low_39',
       '95percent_ci_high_39', 'percent_female', 'num_rural', 'percent_rural'],
      dtype='object', length=507)

In [84]:
county_health.columns[:100]

Index(['fips', 'state', 'county', 'num_deaths',
       'years_of_potential_life_lost_rate', '95percent_ci_low',
       '95percent_ci_high', 'quartile', 'ypll_rate_aian',
       'ypll_rate_aian_95percent_ci_low', 'ypll_rate_aian_95percent_ci_high',
       'ypll_rate_asian', 'ypll_rate_asian_95percent_ci_low',
       'ypll_rate_asian_95percent_ci_high', 'ypll_rate_black',
       'ypll_rate_black_95percent_ci_low', 'ypll_rate_black_95percent_ci_high',
       'ypll_rate_hispanic', 'ypll_rate_hispanic_95percent_ci_low',
       'ypll_rate_hispanic_95percent_ci_high', 'ypll_rate_white',
       'ypll_rate_white_95percent_ci_low', 'ypll_rate_white_95percent_ci_high',
       'percent_fair_or_poor_health', '95percent_ci_low_2',
       '95percent_ci_high_2', 'quartile_2',
       'average_number_of_physically_unhealthy_days', '95percent_ci_low_3',
       '95percent_ci_high_3', 'quartile_3',
       'average_number_of_mentally_unhealthy_days', '95percent_ci_low_4',
       '95percent_ci_high_4', 'quar

There are 507 columns. To reiterate my proposed course of action, we want to first get rid of many different measurements of the same quality and only keep the rates. We also want to remove some redundant columns such as population. The purpose is to hopefully keep the complexity under a managable level, while maintaining the values.

In [89]:
excluded_column_words = [
    'quartile',
    'ci_high',
    'ci_low',
    'fips',
    'num',
    'denominator',
    'ratio',
    'population',
]

In [92]:
filtered_columns = county_health.columns[~county_health.columns.str.contains('|'.join(excluded_column_words))]

In [96]:
print(str(len(filtered_columns)) + ' columns remain!')

190 columns remain!


In [97]:
filtered_county_health = county_health[filtered_columns]

Next, let's merge the health data into the infection data, and check out the merged data.

In [100]:
county = grouped_county.merge(
    filtered_county_health, left_on=['county', 'state'], right_on=['county', 'state']
)

In [103]:
county

,state,county,population,density_km,state_house_blue_perc,days_counted,case_sum,death_sum,case_count_45_days,death_count_45_days,...,percent_65_and_over,percent_black,percent_american_indian_alaska_native,percent_asian,percent_native_hawaiian_other_pacific_islander,percent_hispanic,percent_non_hispanic_white,percent_not_proficient_in_english,percent_female,percent_rural
0,Alabama,Autauga,54571,35.436,22.86,46,67,4,61,3,...,15.562670,19.343177,0.480207,1.224798,0.111509,2.965774,74.308016,0.820225,51.448715,42.002162
1,Alabama,Baldwin,182265,44.261,22.86,56,208,5,168,3,...,20.443350,8.783976,0.772399,1.150343,0.066966,4.646779,83.111337,0.543517,51.538377,42.279099
2,Alabama,Blount,57322,34.325,22.86,45,44,0,44,0,...,18.236515,1.462656,0.653527,0.319848,0.121024,9.571231,86.886238,1.724520,50.726141,89.951502
3,Alabama,Butler,20947,10.411,22.86,45,162,3,162,3,...,20.299797,44.557927,0.376016,1.316057,0.050813,1.509146,51.255081,0.494155,53.429878,71.232157
4,Alabama,Calhoun,118572,75.572,22.86,52,123,3,93,3,...,17.717476,20.850215,0.539916,0.964324,0.110258,3.910673,72.024992,0.991376,51.946586,33.696826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,Wyoming,Natrona,78621,5.684,10.00,50,48,0,48,0,...,15.395311,1.161600,1.466220,0.827909,0.098591,8.581179,86.599254,0.731123,49.617645,14.449304
1458,Wyoming,Park,28702,1.596,10.00,53,1,0,1,0,...,23.206247,0.682035,0.944619,0.821852,0.057973,5.623380,90.915291,0.627835,50.194380,44.240383
1459,Wyoming,Sheridan,29596,4.529,10.00,59,16,0,16,0,...,20.993616,0.797142,1.366057,0.823603,0.112460,4.342937,91.482817,0.348727,49.793272,35.478775
1460,Wyoming,Sweetwater,45267,1.676,10.00,46,19,0,19,0,...,12.111217,1.145153,1.544680,1.010429,0.157952,16.083250,79.312908,1.633221,48.544749,10.916313


We have a lot of columns. Perhaps a lot of them have missing data for more than half of the data. We have no reasonable and accessible way of dealing with missing data here. We could fill in missing values from nearby counties, but that could be both erroneous and difficult. As a result, we will simply get rid of missing data in terms of columns and rows. Let's deal with columns first because we want to keep as many as rows as possible.

In [115]:
# Let's see the columns at near 90% cutoff points
county.dropna(thresh=1300, axis=1).info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1462 entries, 0 to 1461
Data columns (total 90 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   state                                               1462 non-null   object 
 1   county                                              1462 non-null   object 
 2   population                                          1462 non-null   int64  
 3   density_km                                          1462 non-null   float64
 4   state_house_blue_perc                               1462 non-null   float64
 5   days_counted                                        1462 non-null   int64  
 6   case_sum                                            1462 non-null   int64  
 7   death_sum                                           1462 non-null   int64  
 8   case_count_45_days                                  1462 non-null   int64  
 9

At the 90% row number cutoff point, it seems like we have a managable amount of columns. But we still have a few columns with disproportionately more missing data than the rest. Some examples are the suicide rate, firearm fatality rate, and crude rate. With that said, we will set the cutoff point at 1390 rows(95%) to keep the column, percent_enrolled_in_free_or_reduced_lunch, because that information seems useful. Given that I am not a domain export, I hope that decision makes sense.

In [124]:
county.dropna(thresh=1390, axis=1).dropna()

,state,county,population,density_km,state_house_blue_perc,days_counted,case_sum,death_sum,case_count_45_days,death_count_45_days,...,percent_65_and_over,percent_black,percent_american_indian_alaska_native,percent_asian,percent_native_hawaiian_other_pacific_islander,percent_hispanic,percent_non_hispanic_white,percent_not_proficient_in_english,percent_female,percent_rural
0,Alabama,Autauga,54571,35.436,22.86,46,67,4,61,3,...,15.562670,19.343177,0.480207,1.224798,0.111509,2.965774,74.308016,0.820225,51.448715,42.002162
1,Alabama,Baldwin,182265,44.261,22.86,56,208,5,168,3,...,20.443350,8.783976,0.772399,1.150343,0.066966,4.646779,83.111337,0.543517,51.538377,42.279099
2,Alabama,Blount,57322,34.325,22.86,45,44,0,44,0,...,18.236515,1.462656,0.653527,0.319848,0.121024,9.571231,86.886238,1.724520,50.726141,89.951502
3,Alabama,Butler,20947,10.411,22.86,45,162,3,162,3,...,20.299797,44.557927,0.376016,1.316057,0.050813,1.509146,51.255081,0.494155,53.429878,71.232157
4,Alabama,Calhoun,118572,75.572,22.86,52,123,3,93,3,...,17.717476,20.850215,0.539916,0.964324,0.110258,3.910673,72.024992,0.991376,51.946586,33.696826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,Wyoming,Laramie,94483,13.581,10.00,53,163,1,145,1,...,16.028128,2.417758,1.169981,1.365988,0.134376,14.795506,78.501859,0.823311,49.310944,19.784604
1457,Wyoming,Natrona,78621,5.684,10.00,50,48,0,48,0,...,15.395311,1.161600,1.466220,0.827909,0.098591,8.581179,86.599254,0.731123,49.617645,14.449304
1458,Wyoming,Park,28702,1.596,10.00,53,1,0,1,0,...,23.206247,0.682035,0.944619,0.821852,0.057973,5.623380,90.915291,0.627835,50.194380,44.240383
1459,Wyoming,Sheridan,29596,4.529,10.00,59,16,0,16,0,...,20.993616,0.797142,1.366057,0.823603,0.112460,4.342937,91.482817,0.348727,49.793272,35.478775


We are keeping a decent amount of date. Let's go ahead with that decision.

In [125]:
county = county.dropna(thresh=1390, axis=1).dropna()

# Factor analysis